# Import

# Import and Loading Data


In [14]:
import pandas as pd
import numpy as np
import re

pd.set_option("display.max_columns", None)
pd.set_option("display.width", 120)

DATA_PATH = "../data/DataScientist.csv"
df = pd.read_csv(DATA_PATH)

df.shape

(3909, 17)

# Data cleaning and preprocesing

In [15]:
df.columns = (
    df.columns
    .str.strip()
    .str.lower()
    .str.replace(" ", "_")
)

df.columns.tolist()

['unnamed:_0',
 'index',
 'job_title',
 'salary_estimate',
 'job_description',
 'rating',
 'company_name',
 'location',
 'headquarters',
 'size',
 'founded',
 'type_of_ownership',
 'industry',
 'sector',
 'revenue',
 'competitors',
 'easy_apply']

In [16]:
df.info()
(df.isna().mean().sort_values(ascending=False) * 100).round(2)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3909 entries, 0 to 3908
Data columns (total 17 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   unnamed:_0         3909 non-null   int64  
 1   index              3909 non-null   int64  
 2   job_title          3909 non-null   object 
 3   salary_estimate    3909 non-null   object 
 4   job_description    3909 non-null   object 
 5   rating             3909 non-null   float64
 6   company_name       3909 non-null   object 
 7   location           3909 non-null   object 
 8   headquarters       3909 non-null   object 
 9   size               3909 non-null   object 
 10  founded            3909 non-null   int64  
 11  type_of_ownership  3909 non-null   object 
 12  industry           3909 non-null   object 
 13  sector             3909 non-null   object 
 14  revenue            3909 non-null   object 
 15  competitors        3909 non-null   object 
 16  easy_apply         3909 

unnamed:_0           0.0
size                 0.0
competitors          0.0
revenue              0.0
sector               0.0
industry             0.0
type_of_ownership    0.0
founded              0.0
headquarters         0.0
index                0.0
location             0.0
company_name         0.0
rating               0.0
job_description      0.0
salary_estimate      0.0
job_title            0.0
easy_apply           0.0
dtype: float64

In [17]:
# Map possible column names -> our standard names
col_map = {}

# job title
for c in ["job_title", "jobtitle", "title", "position"]:
    if c in df.columns:
        col_map[c] = "job_title"
        break

# job description
for c in ["job_description", "description", "jobdesc", "job_description_text"]:
    if c in df.columns:
        col_map[c] = "job_description"
        break

# salary estimate
for c in ["salary_estimate", "salary", "salaryestimate"]:
    if c in df.columns:
        col_map[c] = "salary_estimate"
        break

# company name
for c in ["company_name", "company", "employer"]:
    if c in df.columns:
        col_map[c] = "company_name"
        break

# location
for c in ["location", "job_location", "city"]:
    if c in df.columns:
        col_map[c] = "location"
        break

df = df.rename(columns=col_map)

# sanity check
needed = ["job_title", "job_description"]
missing_needed = [c for c in needed if c not in df.columns]
missing_needed

[]

In [18]:
df = df.dropna(subset=["job_title", "job_description"]).copy()
df.shape

(3909, 17)

In [19]:
if "salary_estimate" in df.columns:
    # remove missing markers
    df["salary_estimate"] = df["salary_estimate"].astype(str)
    df = df[df["salary_estimate"].str.strip().ne("-1")].copy()

    def parse_salary_k(s):
        s = s.lower()
        s = re.sub(r"\(.*?\)", "", s)          # remove parentheses text
        s = s.replace("$", "").replace(",", "")
        s = s.replace("k", "").strip()

        # keep patterns like "70-120" or "70 - 120"
        m = re.search(r"(\d+)\s*-\s*(\d+)", s)
        if not m:
            return np.nan, np.nan

        return float(m.group(1)), float(m.group(2))

    mins, maxs = [], []
    for s in df["salary_estimate"]:
        mn, mx = parse_salary_k(s)
        mins.append(mn); maxs.append(mx)

    df["min_salary_k"] = mins
    df["max_salary_k"] = maxs
    df["avg_salary_k"] = (df["min_salary_k"] + df["max_salary_k"]) / 2

    # drop rows where salary couldn't be parsed
    df = df.dropna(subset=["avg_salary_k"]).copy()

df.shape

(3909, 20)

In [20]:
if "company_name" in df.columns:
    df["company_name"] = df["company_name"].astype(str)

    # remove trailing rating patterns like "CompanyName 3.9"
    df["company_name_clean"] = df["company_name"].str.replace(r"\s+\d\.\d$", "", regex=True).str.strip()
else:
    df["company_name_clean"] = "Unknown"

In [21]:
def job_family(title: str) -> str:
    t = str(title).lower()

    if "data scientist" in t:
        return "data_scientist"
    if "data engineer" in t:
        return "data_engineer"
    if "data analyst" in t or "business analyst" in t:
        return "data_analyst"
    if "machine learning" in t or "ml engineer" in t:
        return "ml_engineer"
    if "software engineer" in t or "backend" in t or "frontend" in t or "full stack" in t:
        return "software_engineer"
    if "product" in t:
        return "product"
    if "devops" in t or "site reliability" in t or "sre" in t:
        return "devops"
    return "other"

df["job_family"] = df["job_title"].apply(job_family)
df["job_family"].value_counts().head(15)

job_family
other                1252
data_scientist        984
data_analyst          750
data_engineer         742
ml_engineer           141
product                29
software_engineer      11
Name: count, dtype: int64

In [22]:
def seniority(title: str) -> str:
    t = str(title).lower()

    if any(x in t for x in ["intern", "trainee"]):
        return "intern"
    if any(x in t for x in ["junior", "jr", "entry"]):
        return "junior"
    if any(x in t for x in ["senior", "sr", "staff", "principal"]):
        return "senior"
    if any(x in t for x in ["lead", "head"]):
        return "lead"
    if "manager" in t:
        return "manager"
    return "regular"

df["seniority"] = df["job_title"].apply(seniority)
df["seniority"].value_counts()

seniority
regular    2795
senior      823
lead        114
manager      90
junior       55
intern       32
Name: count, dtype: int64

# Text cleaning 

In [23]:
def basic_clean_text(s: str) -> str:
    s = str(s).lower()
    s = re.sub(r"\s+", " ", s)               # normalize whitespace
    s = re.sub(r"[^a-z0-9\+\#\.\s]", " ", s) # keep some tech symbols (+, #, .)
    s = re.sub(r"\s+", " ", s).strip()
    return s

df["job_description_clean"] = df["job_description"].apply(basic_clean_text)

df[["job_title", "job_description_clean"]].head(3)

,job_title,job_description_clean
0,Senior Data Scientist,about hopper at hopper we re on a mission to make booking travel faster easier and more transparent. we are leveraging the power that comes from combining massive amounts of data and machine learning to build the world s fastest growing travel app one that enables our customers to save money and...
1,"Data Scientist, Product Analytics",at noom we use scientifically proven methods to help our users create healthier lifestyles and manage important conditions like type ii diabetes obesity and hypertension. our engineering team is at the forefront of this challenge solving complex technical and ux problems on our mobile apps that ...
2,Data Science Manager,decode m https www.decode m.com data science manager job description we re hiring a data science manager in our new york office to lead decode m s data science team in developing new solutions for our clients and advancing the science of momentum through our proprietary product. this position ba...


# Clean Copy 

In [25]:
df.shape
df.isna().mean().sort_values(ascending=False).head(15)
OUT_PATH = "../data/jobs.csv"
df.to_csv(OUT_PATH, index=False)

print("Saved cleaned dataset to:", OUT_PATH)

Saved cleaned dataset to: ../data/jobs.csv
